In [5]:
import formulallm.formula as f

In [ ]:
f.help()

In [ ]:
code = f.load("./data/parser/TinyDemoParser.4ml")

In [ ]:
f.details("TinyDemoParser")


In [ ]:
f.query("m", "isDone")

In [ ]:
code = f.load("./data/parser/GeneratedTar.4ml")


In [ ]:
f.query("wrongModel", "isDone")

In [ ]:
f.list()